# Import Modules:

In [11]:
import numpy as np
from warnings import warn
import scipy.optimize
import matplotlib.pyplot as plt
# from mpl_toolkits import mplot3d
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib notebook

# %% Define Functions:

sun_mass = 2e33
sun_radius = 7e10
kappa = 3.15e12
G = 6.674e-8


def gauss_p(sa,sb):
    ni, nj = sa.shape
    x = np.zeros(ni)
    if ni != nj:
        print('not rectangular matrix')
        return x
    else:
        n = ni
    if n != sb.shape[0]:
        print('no vector b')
        return x
    a = np.zeros([n,n])
    a[:, :] = sa[:]
    b = np.zeros(n)
    b[:] = sb[:]

    eps = np.finfo(np.float).eps * np.amax(np.abs(np.diag(a)))
    for j in range(0, n - 1):
        if abs(a[j, j]) < eps * 1e5:
            if np.amax(np.abs(a[j + 1:n, j])) < eps:
                print('zero pivot')
                return x
            loc = np.argmax(np.abs(a[j + 1:n, j])) + 1
            if j == 0:
                a[j:j + 1 + loc:loc, j:] = a[j + loc::-loc, j:]
                b[j:j + 1 + loc:loc] = b[j + loc::-loc]
            else:
                a[j:j + 1 + loc:loc, j:] = a[j + loc:j - 1:-loc, j:]
                b[j:j + 1 + loc:loc] = b[j + loc:j - 1:-loc]
        for i in range(j + 1, n):
            p = a[i, j] / a[j, j]
            a[i, j + 1:n] -= p * a[j, j + 1:n]
            b[i] -= p * b[j]
    for i in range(n - 1, -1, -1):
        a_i_i = a[i,i]
        b_i_mone = (b[i] - np.sum(a[i, i + 1:n] * x[i + 1:n]))
        if abs(a_i_i) < eps:
            if abs(b_i_mone) < eps:
                x[i] = 0
                warn('Infinite possible values for x[%s], setting it to 0' % i)
            if abs(b_i_mone) >= eps:
                warn('Unsolvable system of equation - No solution. Returning None')
                return None
        else:
            x[i] = (b[i] - np.sum(a[i, i + 1:n] * x[i + 1:n])) / a[i, i]
    return x


def all_plot(data, type, *args):
    # Plots progress of the root function
    global fig, ax
    # For question 2 - Plot the two points with the style specified in *args:
    if type == '2d function':
        for a in ax:
            a.plot(data[0], data[1] , *args)
    # if the tyep is star, plot the star density graph and the outer radius progression:
    elif type == 'star':
        rs = data['r']
        outer_radius = rs[-1]
        rs = rs / np.max(rs)
        ax[0].clear()
        ax[0].set_xlim((-1.2, 1.2))
        ax[0].set_ylim((-1.2, 1.2))
        jet = plt.get_cmap('viridis_r')
        cNorm = colors.Normalize(vmin=np.min(rs[::10]), vmax=np.max(rs[::10]))
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
        for r in reversed(rs[::10]):
            colorVal = scalarMap.to_rgba(r)
            circle = plt.Circle((0, 0), r, color=colorVal)
            ax[0].add_artist(circle)
        global line1
        current_data = line1.get_data()
        line1.set_data(np.append(current_data[0], current_data[0][-1]+1), np.append(current_data[1], outer_radius))
        ax[1].set_xlim((1, current_data[0][-1]+1.5))
        ax[1].set_ylim((0, max(current_data[1])*1.1))


    # fig.show()
    fig.canvas.draw()


def f(F_value):
    return (1/2) * np.dot(F_value, F_value)


def cube_guess(g, g_0, g_0_der, l_1, g_l_1, l_2, g_l_2):
    mat = np.array([
                    [1/l_1**2, -1/l_2**2],
                    [-l_2/l_1**2, l_1/l_2**2]
                  ])
    vec = np.array([g_l_1-g_0_der*l_1 - g_0,
                    g_l_2-g_0_der*l_2 - g_0
                  ])
    coeffs = ( 1/(l_1 - l_2) ) * ( np.dot(mat, vec) )
    a = coeffs[0]
    b = coeffs[1]

    l_new = (-b+np.sqrt(b**2 - 3*a*g_0_der)) / (3*a)
    l_new = max(l_new, 1/2)
    l_new = min(l_new, 0.1)
    g_l_new = g(l_new)

    if g_l_new <= g_0 + 1e-4 * g_0_der * l_new:
        return l_new, g_l_new
    else:
        return cube_guess(g, g_0, g_0_der, l_new, g_l_new, l_1, g_l_1)


def find_1d_min(g, g_0, g_0_der, g_1):
    # x = scipy.optimize.fminbound(g, 0, 1)
    # return x, g(x)
    guess = - g_0_der / (2 * (g_1 - g_0 - g_0_der))
    guess = max(guess, 0.1)
    g_guess = g(guess)
    if g_guess <= g_0 + 1e-4 * g_0_der * guess:
        return guess, g_guess
    else:
        return cube_guess(g, g_0, g_0_der, guess, g_guess, 1, g_1)

# Test 1d_min_search
# fig, ax = plt.subplots()
# g = lambda x: np.sin(4*x - 4.5) + 0.1*(4*x)**2
# g_0 = g(0)
# g_0_der = 4*np.cos(-4.5)
# g_1 = g(1)
# xs = np.linspace(0, 1, 101)
# ys = g(xs)
# ax.plot(xs, ys)
# fig.show()
# find_1d_min(g, g_0, g_0_der, g_1)


def fix_ordered_coeff(x, step, boundary):
    x = np.insert(x, 0, 0)
    step = np.insert(step, 0, 0)
    x_new = x + step
    if min(x_new[2:] - x_new[1:-1]) >= boundary*0.999:
        return x_new[1:], step[1:]
    else:
        idx = np.argmin((x_new[1:] - x_new[:-1]))#  / (x[1:] - x[:-1]))
        c = (x[idx + 1] - x[idx] - boundary) / (step[idx] - step[idx + 1])
        if min(x[2:] + c*step[2:] - x[1:-1] - step[1:-1]) < boundary*0.999:
            print('Something Wrong - debbug me')
        return fix_ordered_coeff(x[1:], c*step[1:], boundary)


def find_root_nd(F, dF, x, eps, boundary=None, visualization_type=None, level=0):
    level +=1
    F_current = F(x)
    f_current = f(F_current)
    if visualization_type == 'star':
        all_plot({'f': f_current, 'F': F_current, 'r': x}, visualization_type, 'r+')
    J = dF(x)
    step = -gauss_p(J, F_current)
    x_new = x + step
    if boundary is not None:
        x_new, step = fix_ordered_coeff(x, step, boundary)
    if visualization_type == '2d function':
        all_plot(x_new, visualization_type, 'r+')
    F_new = F(x_new)
    f_new = f(F_new)
    if f_new > f_current:
        g = lambda t: f(F(x + t*step))
        g_0_der = np.dot(np.dot(F_current, J), step)
        t, f_new = find_1d_min(g, f_current, g_0_der, f_new)
        x_new = x + t * step
        if visualization_type == '2d function':
            all_plot(x_new, visualization_type, 'mo')
    if (f_new < eps) or (level == 15):
        return x_new
    else:
        return find_root_nd(F, dF, x_new, eps, boundary, visualization_type, level=level)


def gigantic_coeff(m, G, gamma, kappa, n):
    indices = np.linspace(1, n, n)
    # indices[-1] = 0.5*indices[-1]
    C = 3**gamma * (4*np.pi)**(1-gamma) * kappa * m**(gamma-2)/ (G * indices)
    C[-1] = 2 * C[-1]
    return C


def create_initial_rs(R, n):
    return np.linspace(R/n, R, n)


def initiate_start_problem(n, M, R, gamma):
    M = M * sun_mass
    R = R * sun_radius
    m = M / n
    C = gigantic_coeff(m, G, gamma, kappa, n)


    def F(r):
        r_i_p1 = np.append(r[1:], r[-1] + 1)  # The last value is a dummy value and will not have effect on the results.
        r_i_m1 = np.insert(r[:-1], 0, 0)
        special_nth_filter = np.append(np.ones(n - 1), 0)
        F_value = 1 + C * r**4 * (
                                 (r_i_p1**3 - r**3)**(-gamma) * special_nth_filter - (r**3 - r_i_m1**3)**(-gamma)
                                )
        return F_value


    def dF(r):
        # Calculate derivatives of F_{n} separately as they have different fourmula::
        dFn_drn = C[-1] * r[-1]**3 * (r[-1]**3 - r[-2]**3)**(-gamma) * (3*gamma*r[-1]**3 / (r[-1]**3 - r[-2]**3) - 4)
        dFn_drn_m1 = -3*gamma * C[-1] * r[-2]**2 * r[-1] **4 * (r[-1]**3 - r[-2]**3)**(-gamma-1)
        # Calculate elements where i <!=> n:
        # lead of the vector: r_{i+1}, i\in{2..n} (1..n-1 in python)
        r_i_p1 = r[1:]
        # lag of the vector: r_{i-1}, i\in{0..n-2} (-1..n-3 in python)
        r_i_m1 = np.insert(r[:-2], 0, 0)
        # For the next calculations as we calculated already dFn_dr, we dont want r_{n} in the vector:
        r_i = r[:-1]
        C_i = C[:-1]
        # r_{i+1}^{3}-r_{i}^{3} and r_{i}^{3}-r_{i-1}^{3} are common and therefore calculated once here:
        delta_cubed_p1 = r_i_p1**3 - r_i**3
        delta_cubed_m1 = r_i**3 - r_i_m1**3


        dFi_dri = C_i * r_i**3 * (
                          4*(delta_cubed_p1**(-gamma) - delta_cubed_m1**(-gamma)) +
                          3*gamma * r_i**3 * (
                                              delta_cubed_p1 ** (-gamma - 1) + delta_cubed_m1 ** (-gamma - 1)
                                             )
                                  )
        dFi_dri_m1 = -3*C_i * gamma * r_i_m1**2 * r_i**4 * delta_cubed_m1**(-gamma - 1)
        # Ignore dF_1 / dr_0:
        dFi_dri_m1 = dFi_dri_m1[1:]
        dFi_dri_p1 = -3*C_i * gamma * r_i_p1**2 * r_i**4 * delta_cubed_p1**(-gamma - 1)

        J = np.pad(np.diag(dFi_dri_m1), ((1, 1), (0, 2))) + \
            np.pad(np.diag(dFi_dri_p1), ((0, 1), (1, 0))) + \
            np.pad(np.diag(dFi_dri), (0, 1))
        J[-1, -1] = dFn_drn
        J[-1, -2] = dFn_drn_m1

        return J

    r_initial = create_initial_rs(R, n)
    return F, dF, r_initial



# Question 2

In [12]:
def F(x):
    return np.array([x[0]**2 +2*x[1] - 6, -x[0]**2+5*x[1]**3-1])


def dF(x):
    return np.array([[2*x[0], 2], [-2*x[0], 15*x[1]**2]])


xlist = np.linspace(-5, 5, 200)
ylist = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(xlist, ylist)
F0 = X**2 + 2*Y - 6
F1 = -X**2 + 5*Y**3 - 1

# fig, ax = plt.subplots(1,2, subplot_kw={'projection': '3d'})
# ax[0].plot_surface(X, Y, F0,cmap='viridis', edgecolor='none')
# ax[0].set_title('F0')
# ax[1].plot_surface(X, Y, F1,cmap='viridis', edgecolor='none')
# ax[1].set_title('F1')
# ax[0].plot([])
# plt.show()
#
# fig, ax = plt.subplots(1,2, subplot_kw={'projection': '3d'})
fig, ax = plt.subplots(1,2)
ax[0].contourf(X, Y, F0,cmap='viridis', levels = 20)
ax[0].set_title('F0')
ax[1].contourf(X, Y, F1,cmap='viridis', levels = 20)
ax[1].set_title('F1')
ax[0].plot(2, 1, 'bo')
ax[1].plot(2, 1, 'bo')
plt.show()

eps = 0.01
x_initial = np.array([0.4, 0.3])
all_plot(x_initial, '2d function', 'y+')

s = find_root_nd(F, dF, x_initial, eps, visualization_type='2d function')

d = scipy.optimize.fsolve(F, x_initial)
print(s, '\n', d)


<IPython.core.display.Javascript object>

[2.00879975 1.0067654 ] 
 [2. 1.]


<IPython.core.display.Javascript object>

# Question 3

In [ ]:
M = 1
gamma = 5/3
n = 100
R = 1
boundary = 1e5
eps = 0.001

fig, ax = plt.subplots(1,2)
ax[0].set_title('Density Visualization')
ax[1].set_title('Outer Radius')
line1, = ax[1].plot(0, R*sun_radius, 'b-')
plt.show()

F_star, dF_star, r_initial = initiate_start_problem(n, M, R, gamma)
rs = find_root_nd(F_star, dF_star, r_initial, eps, boundary, 'star')
rs = np.insert(rs, 0, 0)
m = M/n * sun_mass
ro = m / (4*np.pi / 3 * (rs[1:]**3 - rs[:-1]**3))
fig_5, ax5 = plt.subplots()
ax5.plot(rs[:-1], ro)
ax5.set_xlabel(r'$r\left[cm^{3}\right]$', usetex=True)
ax5.set_ylabel(r'$\rho\left[\frac{gr}{cm^{3}}\right]$', usetex=True)
ax5.set_title(r'density as a function of the radius', usetex=True)
fig_5.show()

# Question 6

In [ ]:
k = 8
M = 1
gamma = 5/3
n = 100
R = 0.01
boundary = 1e5
eps = 0.001

R_s = np.empty(k)
Ms = np.linspace(0.5, 5, k)

for i, M in enumerate(Ms):
    F_star, dF_star, r_initial = initiate_start_problem(n, M, R, gamma)
    rs = find_root_nd(F_star, dF_star, r_initial, eps, boundary, visualization_type=None)
    R_s[i] = rs[-1]
    print (R_s[i])

fig_6, ax6 = plt.subplots()
ax6.set_xlabel(r'$M$', usetex=True)
ax6.set_ylabel(r'$RM^{\frac{1}{3}}$', usetex=True)
ax6.set_title(r'$RM^{\frac{1}{3}}$ trend', usetex=True)
y_values = R_s * (Ms*sun_mass)**(1/3)
ax6.set_ylim((0, max(y_values) * 1.1))
ax6.plot(Ms*sun_mass, y_values)
fig_6.show()


# Question 7

In [ ]:
k=7
gamma = 1.334
n = 100
R = 1
boundary = 1e2
eps = 0.001
R_s = np.empty(k)
Ms = np.linspace(1.4, 1.46, k)

fig, ax = plt.subplots(1,2)
ax[0].set_title('Density Visualization')
ax[1].set_title('Outer Radius')
line1, = ax[1].plot(0, R*sun_radius, 'b-')
plt.show()

for i, M in enumerate(Ms):
    F_star, dF_star, r_initial = initiate_start_problem(n, M, R, gamma)
    rs = find_root_nd(F_star, dF_star, r_initial, eps, boundary, visualization_type='star')
    R_s[i] = rs[-1]
    print (R_s[i])

fig_7, ax7 = plt.subplots()
ax7.set_xlabel(r'$M$', usetex=True)
ax7.set_ylabel(r'$RM^{\frac{1}{3}}$', usetex=True)
ax7.set_title(r'$RM^{\frac{1}{3}}$ trend', usetex=True)
y_values = R_s * (Ms*sun_mass)**(1/3)
ax7.set_ylim((0, max(y_values) * 1.1))
ax7.plot(Ms*sun_mass, y_values)
fig_7.show()
